# Create Dataset

Helps create a dataset of images captured using `jetcam`

In [1]:
import os
from time import strftime, gmtime
import traitlets
from ipywidgets import Dropdown, Button, Text, Image, VBox, HBox, Label, ToggleButtons, IntText, GridspecLayout, HTML
from IPython.display import display
from pathlib import Path

import cv2

from jetcam.usb_camera import USBCamera
from jetcam.utils import bgr8_to_jpeg

In [2]:
!mkdir -p $data_dir_txt.value

In [3]:
def grid(widgets, labels, **kwargs):
    rows = len(widgets)
    grid = GridspecLayout(rows, 2, **kwargs)
    for i in range(rows):
        grid[i, 0] = Label(labels[i])
        grid[i, 1] = widgets[i]
    return grid

### Create Widgets

In [4]:
camera = None
data_root_path = None
API_PREFS = [('Any', cv2.CAP_ANY), 
             ('Gstreamer', cv2.CAP_GSTREAMER)]
CATEGORY_LABELS = [('Seeing', 'see'), ('Looking Away', 'away')]

In [5]:
# Left panel
data_folder_widget = Text(value='data', description_tooltip='Folder where data will be stored')
api_pref_widget = Dropdown(options=API_PREFS)
height_widget = IntText(value=480)
width_widget = IntText(value=640)
fps_widget = IntText(value=30)
settings_grid = grid([data_folder_widget, api_pref_widget, height_widget, width_widget, fps_widget],
                     ['Data Folder', 'API Preference', 'Capture Height', 'Capture Width', 'FPS'],
                     width='450px')
settings_label = HTML(value='<b>Settings</b>')
apply_settings_widget = Button(description='Apply Settings')

hr = HTML(value='<hr/>')

category_widget = Dropdown(options=CATEGORY_LABELS)
session_widget = Text(description_tooltip='Optional folder to group images in a session')
camera_toggle = ToggleButtons(options=[('Live Camera', True), ('Stop', False)], value=False)
data_info_grid = grid([category_widget, session_widget],
                      ['Category', 'Session Label'],
                      width='450px')
left_panel = VBox([settings_label, settings_grid, apply_settings_widget,
                   hr,
                   data_info_grid, camera_toggle])

# Right panel
camera_widget = Image(width=400, height=300, format='jpeg')

save_image_button = Button(description='Save Image', tooltip='Click photo')
right_panel = VBox([camera_widget, save_image_button])

all_panels = HBox([left_panel, right_panel])

In [6]:
def apply_settings(c):
    global camera
    global data_root_path
    data_root_path = Path(data_folder_widget.value)
    data_root_path.mkdir(exist_ok=True)
    capture_height = height_widget.value
    capture_width = width_widget.value
    fps = fps_widget.value
    if camera is not None:
        camera.unobserve_all()
        camera.release_camera()
    camera = USBCamera(width=400,
                       height=300,
                       capture_height=capture_height,
                       capture_width=capture_width,
                       capture_fps=fps,
                       capture_device=0)
    traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
apply_settings_widget.on_click(apply_settings)

def camera_toggled(change):
    global camera
    if change['new']:
        camera.running = True
    else:
        camera.running = False
camera_toggle.observe(camera_toggled, names='value')

def image_dir_path():
    global data_root_path
    path = data_root_path / category_widget.value
    if session_widget.value is not None:
        path = path / session_widget.value
    path.mkdir(exist_ok=True)
    return path

def save_image(c):
    file_name = strftime('%y%m%d%H%M%S.jpg', gmtime())
    image_path = image_dir_path() / file_name
    cv2.imwrite(str(image_path.absolute()), camera.value)
save_image_button.on_click(save_image)

In [7]:
display(all_panels)